In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
import numpy as np
import warnings
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
warnings.filterwarnings("ignore")

In [3]:
train_df = pd.read_csv("/media/keshav-pc/HDD/data/brain_tumor_train_compact.csv")
test_df = pd.read_csv("/media/keshav-pc/HDD/data/brain_tumor_test_compact.csv")

In [4]:
train_df.head()

,image,label,label_id
0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",glioma,0
1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",glioma,0
2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",glioma,0
3,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",glioma,0
4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",glioma,0


In [5]:
X_train = np.array([np.fromstring(img, sep=",") for img in train_df["image"]])
y_train = train_df["label_id"].values

X_test = np.array([np.fromstring(img, sep=",") for img in test_df["image"]])
y_test = test_df["label_id"].values

In [6]:
X_train = X_train.reshape(-1, 150, 150, 3)
X_test = X_test.reshape(-1, 150, 150, 3)

In [7]:
print(X_train.shape)
print(y_train.shape)

(5712, 150, 150, 3)
(5712,)


In [8]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,
    Dense,
    Flatten,
    Conv2D,
    Reshape,
    MaxPooling2D,
    AveragePooling2D,
    BatchNormalization,
    Dropout,
)
import numpy as np
import ast
from tensorflow.keras import metrics


class BinaryF1Score(metrics.Metric):
    def __init__(self, name="f1_score", threshold=0.5, **kwargs):
        super().__init__(name=name, **kwargs)
        self.threshold = threshold
        self.precision = metrics.Precision(thresholds=threshold)
        self.recall = metrics.Recall(thresholds=threshold)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred)
        self.recall.update_state(y_true, y_pred)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()


def get_metric_mapping():
    """Returns all available metrics without validation"""
    return {
        # Universal metrics (work for any task)
        "mae": "mae",
        "mse": "mse",
        "accuracy": "accuracy",
        # Classification metrics
        "precision": metrics.Precision(name="precision"),
        "recall": metrics.Recall(name="recall"),
        "f1_score": BinaryF1Score(name="f1_score"),
    }


def resolve_metrics(metric_names):
    """Simple metric name to implementation mapping"""
    mapping = get_metric_mapping()
    return [mapping[name] for name in metric_names if name in mapping]


def handle_error(error):
    error_message = f"An error occurred: {error}"
    print(error_message)


class CustomCNN:
    def __init__(self, config):
        self.config = config
        self.model = Sequential()
        try:
            input_shape = ast.literal_eval(config.get("input_shape", "(128, 128, 1)"))
            self.model.add(Input(shape=input_shape))

            # Adding layers to the model
            for i in range(len(config["layers"])):
                layer = config["layers"][i]
                layer_type = layer["layer_type"]

                if layer_type == "dense":
                    dense_layer = Dense(
                        int(layer["num_nodes"]),
                        activation=layer.get("activation_function", "relu"),
                    )
                    # Add regularization if specified
                    if "regularizer" in layer:
                        reg_config = layer["regularizer"]
                        if reg_config["type"] == "l2":
                            dense_layer.kernel_regularizer = regularizers.l2(
                                float(reg_config["factor"])
                            )
                    self.model.add(dense_layer)

                elif layer_type == "flatten":
                    self.model.add(Flatten())

                elif layer_type == "convolution":
                    conv_layer = Conv2D(
                        int(layer["filters"]),
                        kernel_size=eval(layer.get("kernel_size", "(3, 3)")),
                        strides=eval(layer.get("stride", "(1, 1)")),
                        activation=layer.get("activation_function", "relu"),
                        padding=layer.get("padding", "same"),
                    )
                    self.model.add(conv_layer)

                elif layer_type == "reshape":
                    self.model.add(Reshape(eval(layer["target_shape"])))

                elif layer_type == "pooling":
                    if layer.get("pooling_type", "max") == "max":
                        self.model.add(
                            MaxPooling2D(
                                pool_size=eval(layer.get("pool_size", "(2, 2)")),
                                strides=eval(
                                    layer.get("stride", "None")
                                ),  # None means same as pool_size
                            )
                        )
                    elif layer.get("pooling_type") == "average":
                        self.model.add(
                            AveragePooling2D(
                                pool_size=eval(layer.get("pool_size", "(2, 2)")),
                                strides=eval(layer.get("stride", "None")),
                            )
                        )

                elif layer_type == "batch_norm":
                    self.model.add(BatchNormalization())

                elif layer_type == "dropout":
                    self.model.add(Dropout(float(layer.get("rate", 0.5))))

            # Output layer
            output_layer = config["output_layer"]
            output_dense = Dense(
                int(output_layer["num_nodes"]),
                activation=output_layer.get("activation_function", "linear"),
            )
            if "regularizer" in output_layer:
                reg_config = output_layer["regularizer"]
                if reg_config["type"] == "l2":
                    output_dense.kernel_regularizer = regularizers.l2(
                        float(reg_config["factor"])
                    )
            self.model.add(output_dense)

            # Compile model
            optimizer_config = config.get(
                "optimizer", {"type": "adam", "learning_rate": 0.001}
            )
            if optimizer_config["type"] == "adam":
                optimizer = optimizers.Adam(
                    learning_rate=float(optimizer_config.get("learning_rate", 0.001))
                )
            elif optimizer_config["type"] == "sgd":
                optimizer = optimizers.SGD(
                    learning_rate=float(optimizer_config.get("learning_rate", 0.01))
                )
            else:
                optimizer = optimizer_config["type"]  # assume it's a string like 'adam'

            metrics = resolve_metrics(config.get("test_metrics", []))
            self.model.compile(
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                optimizer=optimizer,
                metrics=metrics,
            )

        except Exception as e:
            handle_error(e)

    def fit(self, X, y, epochs=1, batch_size=32, validation_data=None, callbacks=None):
        try:
            # Input validation
            if X is None or y is None:
                print("Error: X and y cannot be None")
                return None

            # Check model compilation
            if not hasattr(self.model, "optimizer"):
                print("Error: Model is not compiled")
                return None
            return self.model.fit(
                X,
                y,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=validation_data,
                callbacks=callbacks,
                verbose=0,
            )
        except Exception as e:
            error_message = f"An error occurred in Fit Function: {e}"
            print(error_message)

    def predict(self, X):
        try:
            return self.model.predict(X)
        except Exception as e:
            handle_error(e)

    def get_parameters(self):
        try:
            params = {"weights": []}
            for i, layer in enumerate(self.model.layers):
                layer_weights = layer.get_weights()
                # Convert each weight array to lists (can't do at once.)
                params["weights"].append([w.tolist() for w in layer_weights])
            return params
        except Exception as e:
            handle_error(e)

    def update_parameters(self, new_params):
        try:
            for i, (layer, layer_params) in enumerate(
                zip(self.model.layers, new_params["weights"])
            ):
                # Convert lists back to numpy arrays
                layer.set_weights([np.array(w) for w in layer_params])
        except Exception as e:
            handle_error(e)

    def evaluate(self, x_test, y_test, batch_size=32):
        try:
            if x_test is None or y_test is None:
                print("Error: x_test and y_test cannot be None")
                return None
            
            results = self.model.evaluate(x_test, y_test, batch_size=batch_size)
            # Return a dictionary of metric names and their values
            metrics = ["loss"] + self.config.get("test_metrics", [])
            return dict(zip(metrics, results))
        except Exception as e:
            error_message = f"An error occurred in Evaluate Function: {e}"
            print(error_message)


2025-09-11 12:28:50.340545: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-11 12:28:50.347158: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-11 12:28:50.363476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757573930.385949  320763 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757573930.393175  320763 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757573930.413880  320763 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [9]:
cnn_model = CustomCNN(
    config = {
        "test_metrics": ["accuracy"],   # jo metrics chahiye
        "lr": 0.01,                                   # not actually used, optimizer ka learning_rate use hoga
        "n_iters": 1000,                              # unused placeholder, but keep if needed
        "input_shape": "(150,150,3)",                 # input shape
        "output_layer": {                             # final classification layer
            "num_nodes": "4",                         # 4 classes
            "activation_function": "softmax"          # softmax for multi-class
        },
        "loss": "sparse_categorical_crossentropy",    # sparse loss (labels integer me hone chahiye: 0,1,2,3)
        "optimizer": {
            "type": "adam",
            "learning_rate": "0.001"                  # learning rate 0.2 bahut jyada hai, 0.001/0.0005 better
        },
        "layers": [
            {
                "layer_type": "convolution",
                "filters": "32",
                "kernel_size": "(3,3)",
                "stride": "(1,1)",
                "padding": "same",
                "activation_function": "relu"
            },
            {
                "layer_type": "pooling",
                "pooling_type": "max",
                "pool_size": "(10,10)",
                "stride": "(10,10)"
            },
            {
                "layer_type": "convolution",
                "filters": "32",
                "kernel_size": "(3,3)",
                "stride": "(1,1)",
                "padding": "same",
                "activation_function": "relu"
            },
            {
                "layer_type": "pooling",
                "pooling_type": "max",
                "pool_size": "(2,2)",
                "stride": "(2,2)"
            },
            {
                "layer_type": "convolution",
                "filters": "32",
                "kernel_size": "(3,3)",
                "stride": "(1,1)",
                "padding": "same",
                "activation_function": "relu"
            },
            {
                "layer_type": "flatten"
            }
        ]
    }
)

# make sure y_train is integer labels: shape (5712,)
# example: array([0,2,1,3,...])

cnn_model.fit(X_train, y_train, epochs=2, batch_size=32)

2025-09-11 12:29:00.579069: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-09-11 12:29:04.527421: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1542240000 exceeds 10% of free system memory.


In [10]:
cnn_model.predict(X_test[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


array([[8.5929763e-01, 1.3725710e-01, 1.9868075e-04, 3.2464766e-03]],
      dtype=float32)

In [11]:
# svm_model.update_parameters()
cnn_model.get_parameters()

{'weights': [[[[[[0.049846384674310684,
       -0.04118361324071884,
       -0.06020139530301094,
       0.08660238236188889,
       -0.07646133005619049,
       -0.0959949865937233,
       0.05728066712617874,
       0.015055719763040543,
       -0.07584932446479797,
       -0.11112428456544876,
       -0.1040562316775322,
       -0.06493375450372696,
       -0.005450547207146883,
       -0.011159272864460945,
       -0.057622265070676804,
       0.021753404289484024,
       -0.10270383208990097,
       0.04620083421468735,
       0.07965113967657089,
       0.09831670671701431,
       0.08707195520401001,
       -0.035434871912002563,
       0.004216729197651148,
       0.06944075226783752,
       0.11466164141893387,
       -0.07418815046548843,
       0.020322417840361595,
       0.006152218207716942,
       -0.13075286149978638,
       0.1428016573190689,
       -0.04912646859884262,
       0.007725899573415518],
      [-0.07499291002750397,
       -0.13521060347557068,
       0.1

In [14]:

y_pred = cnn_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
# cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", acc)
print("Confusion Matrix:\n", cm)


 5/41 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step

2025-09-11 12:33:26.791645: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 353970000 exceeds 10% of free system memory.


41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [24]:

metrics = ["accuracy", "precision", "recall", "f1_score", "confusion_matrix"]
results = svm_model.evaluate(X_test, y_test, metrics=metrics)
print("Evaluation Results:", results)


Evaluation Results: {'accuracy': 0.44207138259065876, 'precision': 0.8639377843811727, 'recall': 0.44207138259065876, 'f1_score': 0.5563775118564089, 'confusion_matrix': [[84195, 110280], [6726, 8514]]}
